# Quantum Computation - Week 5

&nbsp;

## Contents

&nbsp;

1. [Easter Assignments (solved)](#easter)

&nbsp;

2. [Quantum State Tomography](#qst)

&nbsp;

#### Module imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# importing Qiskit
from qiskit import Aer, IBMQ
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, transpile

from qiskit.tools.visualization import plot_histogram

#IBMQ.load_account()

import itertools
import math
pi=math.pi

**Quickly simulate circuit**

In [ ]:
# Plot results
def show_results(D):
    # D is a dictionary with classical bits as keys and count as value
    # example: D = {'000': 497, '001': 527}
    plt.bar(range(len(D)), list(D.values()), align='center')
    plt.xticks(range(len(D)), list(D.keys()))
    plt.show()

# Execute circuit, display a histogram of the results
def execute_locally(qc, draw_circuit=False):
    # Compile and run the Quantum circuit on a simulator backend
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim)
    result_sim = job_sim.result()
    result_counts = result_sim.get_counts(qc)
    
    # Print the results
    print("simulation: \n\n", result_counts)
    show_results(result_counts)


## 3. Easter Assignments (solved)<a id='easter'></a>

<div class="alert alert-block alert-warning">

**Assignment A**

&nbsp;

Extend the `grover_3sat` function with `cn_not` to allow for $ n> 3$ literals per clause; assume at most 3 clauses for the formula.

&nbsp;

Notes:
   * the register `f_in` has $n$ qubits for $n$ literals $x_i$;
   * the number of qubits in the `aux` register will also be affected;
   * besides `grover_3sat`, functions `sub_oracle` and `inversion_about_mean` need to be extended.

</div>

**Assignees**
   * Ana Carvalho
   * António Pereira
   * José Guimarães

<div class="alert alert-block alert-info">

**Expanding the XOR-equivalent condition to >3 literals**

&nbsp;

If we have 3 literals, the condition 

$$ y = (x_1 \vee \neg x_2 \vee x_3)$$ 


&nbsp;

expands to:



$$
y = x_1 \oplus \neg x_2 \oplus x_3 \oplus ( x_1 \wedge \neg x_2 \wedge x_3), 
$$

&nbsp;

What if instead we have 4 (or more) literals?

$$ y = (x_1 \vee x_2 \vee x_3 \vee x_4)$$

&nbsp;

Option A:

$$
y = x_1 \oplus x_2 \oplus x_3 \oplus x_4 \oplus ( x_1 \wedge x_2 \wedge x_3 \wedge x_4)
$$

&nbsp;

**Option B**:

$$
y = x_1 \oplus x_2 \oplus x_3 \oplus x_4 \oplus ( x_1 \wedge x_2 \wedge x_3) \oplus ( x_1 \wedge x_3 \wedge x_4)
\oplus ( x_2 \wedge x_3 \wedge x_4)
$$


</div>

In [ ]:
# Resolucao - Antonio Pereira

def cn_not(circuit, ctrls, trg, aux):
    
    N=len(ctrls)
    
    circuit.ccx(ctrls[0],ctrls[1],aux[0])
    
    for i in range(2,N):
        circuit.ccx(ctrls[i],aux[i-2],aux[i-1])
    
    circuit.cx(aux[N-2],trg)
    
    
    for i in range(2,N):
        circuit.ccx(ctrls[N-i+1],aux[N-i-1],aux[N-i])
    
    circuit.ccx(ctrls[0],ctrls[1],aux[0])
    circuit.barrier()
    
def combin(qbits, n):
    qbits = list(qbits)
    for i in itertools.combinations(qbits, n):
        yield i
        
def create_cn_nots(circuit, f_in, f_out, aux):
    l=list(range(3, len(f_in), 2))
    for i in l:
        for j in combin(range(len(f_in)), i):
                cn_not(circuit,[f_in[k] for k in j],f_out,aux)
    
    if (len(f_in) % 2 !=0):
        cn_not(circuit, f_in,f_out, aux)

In [ ]:
# Resolucao - Antonio Pereira (continuacao)

def input_state(circuit, f_in, f_out):
    circuit.h(f_in)
    circuit.x(f_out)
    circuit.h(f_out)
    
def sub_oracle(circuit, f_in, f_outx, aux, clausule):
    for i in clausule:
        if i<0:
            circuit.x(f_in[abs(i)-1])
            
    for i in clausule:
        circuit.cx(f_in[abs(i)-1],f_outx)
    
    circuit.barrier()
    
    create_cn_nots(circuit, f_in, f_outx, aux)
    
    for i in clausule:
        if i<0:
            circuit.x(f_in[abs(i)-1])

        
def inversion_about_mean(circuit, f_in, aux):
    n = f_in.__len__()
    
    circuit.h(f_in)
    circuit.x(f_in)
    
    
    circuit.h(f_in[n-1])
    
    if n == 2:
        circuit.cx(f_in[0], f_in[1])
    
    elif n == 3:
        circuit.ccx(f_in[0], f_in[1], f_in[2])
    
    elif n >= 4:
        cn_not(circuit, [f_in[i] for i in range(n-1)], f_in[n-1], aux)

    
    circuit.h(f_in[n-1])
        
    circuit.x(f_in)
    circuit.h(f_in)


We can visualize the `sub_oracle` function:

In [ ]:
sat_formula = [[1, 2, -3, 4], [-1, -2, -3, 4], [-1, 2, 3, 4]]


# Number of clauses 
k = len(sat_formula)
n = len(sat_formula[0])
# n literals - n qubits

f_in = QuantumRegister(n, 'f_in')

# Number of qubits in f_out: number of clauses + 1
f_out = QuantumRegister(k+1, 'f_out')

# Ancilla qubits
aux = QuantumRegister(n-1, 'aux')

# Measure f_in at the end to a classical register
cr = ClassicalRegister(n)

# Create Circuit
grover = QuantumCircuit(f_in, aux, f_out, cr)

sub_oracle(grover, f_in, f_out[0], aux, sat_formula[0])

grover.draw(output='mpl')

<div class="alert alert-block alert-warning">

**Assignment B**

&nbsp;

Extend the `grover_3sat` function with `cn_not` to allow for $ k > 3$ _clausules per formula;_ assume at most 3 literals per clausule.

&nbsp;

Notes:
   * the register `f_out` has $k+1$ qubits for $k$ clausules;
   * the number of qubits in the `aux` register will also be affected;
   * besides `grover_3sat`, function `black_box_u_f` needs to be extended.

</div>

**Assignees**
   * Alexandra Alves
   * Mafalda Alves
   * Marco Barbosa

In [ ]:
# Resolucao - Alexandra Alves

# Função black_box_u_f - verifica se os estados de um conjunto de qubits satisfazem o problema 3SAT dado por uma fórmula.
#
# Inputs: o circuito quântico circuit, o registo de qubits de entrada f_in, o registo de qubits de saída f_out, o registo de 
#qubits auxiliares aux, e uma fórmula composta por uma lista de cláusulas (que por sua vez consistem em listas de inteiros).
#
# Inverte (na base computacional) o estado do último qubit do registo f_out se os estados de f_in constituem uma solução válida
#(verificando a correção das cláusulas individuais, bem como a conjunção de todas elas). Estando ele no estado de Hadamard |->,
#isto tem o efeito de o colocar no estado -|->, efetivamente invertendo a amplitude do estado de 3 qubits que é solução do 
#problema.
# Caso contrário, deixa-o inalterado. As amplitudes com que os restantes estados (que não correspondem a soluções para 
#o problema) ocorrem não são portanto afetadas.


def black_box_u_f(circuit, f_in, f_out, aux, sat_formula):
    # Verifica-se primeiro a correção de cada cláusula (ser correta implica ser válida e conter apenas um literal verdadeiro).
    for (n, clausule) in enumerate(sat_formula):
        sub_oracle(circuit, f_in, f_out[n], aux, clausule)
        circuit.barrier()
        
    # A palavra dada por f_in apenas deve ser identificada como a solução se todas as cláusulas avaliam a True, ou seja,
    #se o sub_oracle colocou para cada uma o respetivo qubit f_out a |1>.
    cn_not(circuit, f_out[0:len(sat_formula)], f_out[len(sat_formula)], aux)
    circuit.barrier()
    
    # Volta-se a inverter os qubits que se negou, para que estejam inicializados a |0> e possam ser reutilizados em
    #iterações futuras.
    for (n, clausule) in enumerate(sat_formula):
        sub_oracle(circuit, f_in, f_out[n], aux, clausule)
        circuit.barrier()
    


Working `grover_3sat` for $>3$ literals, $>3$ clauses:

In [ ]:

def grover_3sat(sat_formula):
    # Number of clauses 
    k = len(sat_formula)
    
    # N literals - n qubits
    n = len(sat_formula[0])
    
    if (k>n):
        aux_n = k-1
    
    else:
        aux_n = n-1

    f_in = QuantumRegister(n, 'f_in')
    
    # Number of iterations
    iterations = int(round(pi/4*math.sqrt(2**n)))
    
    # Number of qubits in f_out: number of clauses + 1
    f_out = QuantumRegister(k+1, 'f_out')
    
    # Ancilla qubits
    aux = QuantumRegister(aux_n, 'aux')
    
    # Measure f_in at the end to a classical register
    cr = ClassicalRegister(n)
    
    # Create Circuit
    grover = QuantumCircuit(f_in, aux, f_out, cr)
    
    # Initialize
    input_state(grover, f_in, f_out[k])
    
    # Iterate blackbox and inversion
    for i in range(iterations):
        black_box_u_f(grover, f_in, f_out, aux, sat_formula)
        inversion_about_mean(grover, f_in, aux)
    
    grover.measure(f_in, cr)

    execute_locally(grover)

In [ ]:
sat_formula = [[1, 2, -3, 4], [-1, -2, -3, 4], [-1, 2, 3, 4], [-1, 2, 3, 4]]

grover_3sat(sat_formula)

<div class="alert alert-block alert-warning">

**Assignment C**

Implement a so-called _splitting decomposition_ of a $C^nNOT$ gate, which decomposes the $n$ controls into Toffoli gates in a recursive manner, requiring $n-2$ aux qubits:

<img src="img/decompose1.png" width="300 px" align="center">

In the image, a $C^7NOT$ gate is decomposed into $C^4NOT$ gates, which further need to be decomposed - for a $C^4NOT$ gate, the remaining decomposition would be:

<img src="img/decompose2.png" width="500 px" align="center">

Note that each stage of decomposition requires one aditional aux qubit.

</div>

**Assignees**
   * José Marques
   * José Martins
   * Tomás Sousa

In [ ]:
# Resolucao - Tomas Sousa

def recursive_CNOT_split(circuit, ctrls, trg, aux):
    N = len(ctrls)-1
    len_c = len(ctrls)-1
    rec_splitdec_cnot(circuit,ctrls,aux,trg,N,len_c)
    
def rec_splitdec_cnot(circuit,ctrls,aux,trg,N,l_cntrls):
    
    if (N==l_cntrls):
        rec_splitdec_cnot(circuit,ctrls,aux,trg,N-1,l_cntrls)
        circuit.barrier()
        circuit.barrier()
        
        circuit.ccx(ctrls[N],aux[N-2],trg)
        circuit.barrier()
        circuit.barrier()
        
        rec_splitdec_cnot(circuit,ctrls,aux,trg,N-1,l_cntrls)
    
    elif(N==2):
        circuit.ccx(ctrls[0],ctrls[1],aux[0])
        circuit.ccx(ctrls[N],aux[N-2],aux[N-1])
        circuit.ccx(ctrls[0],ctrls[1],aux[0])
        
    else:
        rec_splitdec_cnot(circuit,ctrls,aux,trg,N-1,l_cntrls)
        circuit.barrier()
        circuit.ccx(ctrls[N],aux[N-2],aux[N-1])
        circuit.barrier()
        rec_splitdec_cnot(circuit,ctrls,aux,trg,N-1,l_cntrls)

Visualizing the decomposition:

In [ ]:
# Number of controls
n = 5

# Circuit constructor
controls=QuantumRegister(n, 'controls')
aux=QuantumRegister(n-2, 'aux')
trg=QuantumRegister(1, 'target')
qc=QuantumCircuit(controls, aux, trg)

# Control list 
control_list = []
for i in range(n):
    control_list.append(controls[i])

# Call to cnot function
recursive_CNOT_split(qc, control_list, trg, aux)

qc.draw(output='mpl')

<div class="alert alert-block alert-warning">

**Bonus - Random Assignment Generator**

</div>

In [ ]:
# Resolucao - Tomas Sousa

# '00' - Assignment A, '01' - Assignment B, '11' - Assignment C
assignment = {'00' : 'A', '01' : 'B', '11' : 'C'}
# init quantum state
init_state = np.array([1, 1, 0, 1]) * 1/np.sqrt(3)

qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc = QuantumCircuit(qr, cr)

qc.initialize(init_state, [qr[0], qr[1]])
qc.measure(qr, cr)

# Assignee name 
assignees = ['Ana Filipa Carvalho','Marco Barbosa','António Pereira','José Guimarães','Alexandra Alves','Mafalda Alves','José Marques','José Martins','Tomás Sousa']

# Counter 

counter = {'A':0, 'B':0 ,'C':0}

for i in range(len(assignees)):
    
    if (len(assignment) == 1):
        print(assignees[i]+ ' gets assignment ' + list(assignment)[0] + '\n')
        
    else:
        backend = Aer.get_backend('qasm_simulator')
        job = execute(qc, backend, shots=1)
        qc_state = job.result().get_counts(qc)
        #print(qc_state)
        state_string = list(qc_state.keys())[0]
        counter[assignment[state_string]] = counter.get(assignment[state_string],0) + 1

        while (counter[assignment[state_string]] > 3):
            backend = Aer.get_backend('qasm_simulator')
            job = execute(qc, backend, shots=1)
            qc_state = job.result().get_counts(qc)
            state_string = list(qc_state.keys())[0]
            counter[assignment[state_string]] = counter.get(assignment[state_string],0) + 1

        print(assignees[i]+ ' gets assignment ' + assignment[state_string] + '\n')

# 2. Quantum State Tomography<a id='qst'></a>

&nbsp;

**Tomography** is a method of imaging a three-dimensional object, and its internal structure, through observation of different cross-sections of the object.

<img src="img/tomo.png" width="500 px" align="center">

&nbsp;

Similarly, **quantum state tomography** is the process of reconstructing the density matrix describing a quantum system, through different quantum state measurements of the system.

<img src="img/q_tomo.png" width="500 px" align="center">

&nbsp;

The density matrix allows for the representation of any mixed states, i.e. statistical mixtures of pure states:

$$ \rho = \sum_j p_j |\psi_j \rangle \langle \psi_j| $$

For pure states, $j=1$

**Module imports**

In [ ]:
# Noise simulation
from qiskit.providers.aer import noise

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Interactive tools
import qiskit.tools.jupyter

# Plotting function
from matplotlib import pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-info">
    
**Example** 

How to perform 2-qubit state tomography:

</div>

In [ ]:
# Create the expected density matrix

qr = QuantumRegister(2)
bell = QuantumCircuit(qr)
bell.h(qr[0])
bell.cx(qr[0], qr[1])

bell.draw(output='mpl')

In [ ]:
# Get statevector representation

job = execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

In [ ]:
rho_bell = np.outer(psi_bell, np.conj(psi_bell))
rho_bell

In [ ]:
# Creating the tomography circuits

qst_bell = state_tomography_circuits(bell, [qr[0],qr[1]])

for circuit in qst_bell:
    circuit.draw(output='mpl')

In [ ]:
# Executing and fitting the circuits

job = execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=1000)

tomo_bell = StateTomographyFitter(job.result(), qst_bell)

experimental_bell = tomo_bell.fit()

np.around(experimental_bell, decimals=2)

<div class="alert alert-block alert-info">
    

**Fidelity of quantum states**

&nbsp;

Given two density operators $\rho$ and $\sigma$, the fidelity is generally defined as the quantity 

&nbsp;

$$F(\rho, \sigma) = \left[\operatorname{tr} \sqrt{\sqrt\rho \sigma\sqrt\rho}\right]^2$$

&nbsp;

Or, when comparing the fidelity between two pure states


$$F(\rho, \sigma)=|\langle\psi_\rho|\psi_\sigma\rangle|^2$$

</div>

In [ ]:
# Obtaining reconstructed state fidelity

F_bell = state_fidelity(psi_bell, experimental_bell)
print('Fit Fidelity =', F_bell)

## 2.1 Quantum state tomography with measurement noise

&nbsp;

State reconstruction methods are experimentally useful in the determination of quantum states of physical devices, since we may want to test a device's reliability or suceptibility to noise. 

But how does tomography when the measurement process itself is noisy? 

In [ ]:
IBMQ.load_account()

In [ ]:
%qiskit_backend_overview

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')

backend = provider.get_backend('ibmq_vigo')

backend

<div class="alert alert-block alert-info">

**Noise models**

&nbsp;

**Aer** simulators allow for the specification of custom noise models.

</div>

In [ ]:
# Building a noise model

noise_model = noise.NoiseModel()

for qi in range(qr.__len__()):
    read_err = noise.errors.readout_error.ReadoutError([[0.75, 0.25],[0.1,0.9]])
    noise_model.add_readout_error(read_err,[qi])
    
# Or, for simetric noise

noise_rate = 0.1
noise_model = noise.NoiseModel()

for qi in range(qr.__len__()):
    read_err = noise.errors.readout_error.ReadoutError([[1-noise_rate, noise_rate],[noise_rate,1-noise_rate]])
    noise_model.add_readout_error(read_err,[qi])

In [ ]:
# We can now re-run the simulation by adding the noise model to the execution inistruction:

backend = Aer.get_backend('qasm_simulator')
job_tomo = execute(qst_bell, backend=backend, shots=1000, noise_model=noise_model)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

experimental_bell = tomo_bell.fit()

F_bell = state_fidelity(psi_bell, experimental_bell)
print('Fit Fidelity =', F_bell)

<div class="alert alert-block alert-warning">

**Exercise 1**

&nbsp;

Implement a routine to estimate the fit fidelity of the quantum tomography procedure associated with the `bell` circuit, for measurement noise rates `[0.00, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50]`.

Use the matplotlib `plt` function to plot the fidelity as function of noise rate.

</div>

<div class="alert alert-block alert-warning">

**Exercise 2**

&nbsp;

For a fixed noise rate of $0.2$ implement a routine to estimate the fit fidelity of the quantum tomography procedure associated with the `bell` circuit, for execution shots `[10, 50, 100, 500, 1000, 5000, 10000]`

Use the matplotlib `plt` function to plot the fidelity as function of execution shots.

</div>